In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv(".\\adult.csv")
print(dataset['race'].value_counts())
dataset = dataset.replace({"Black": "Unprivileged",  "Asian-Pac-Islander": "Unprivileged", "Amer-Indian-Eskimo": "Unprivileged", "Other": "Unprivileged", "White": "Privileged"})

# Esse código vai ser feito com Pandas, provavelmente não vai estar muito bem otimizado, mas não vou me preocupar com isso agr.

White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64


In [2]:
# capturando as frequências desejadas pro cálculo das probabilidades esperadas
n = dataset.shape[0]
n_p = dataset[dataset['race'] == "Privileged"].shape[0]
n_unp = dataset[dataset['race'] == "Unprivileged"].shape[0]
n_fav = dataset[dataset['income'] == ">50K"].shape[0]
n_unfav = dataset[dataset['income'] == "<=50K"].shape[0]
# statistical parity -> n_p_fav/n_p == n_unp_fav/n_unp

n_p_fav = dataset[(dataset['race'] == "Privileged") & (dataset['income'] == ">50K")].shape[0]
n_p_unfav = dataset[(dataset['race'] == "Privileged") & (dataset['income'] == "<=50K")].shape[0]
n_unp_fav = dataset[(dataset['race'] == "Unprivileged") & (dataset['income'] == ">50K")].shape[0]
n_unp_unfav = dataset[(dataset['race'] == "Unprivileged") & (dataset['income'] == "<=50K")].shape[0]

In [3]:
#vou calcular das duas formas que mostram -> Expected/Observed e (Np * Nfav)/(N * Np_fav)
w_p_fav = n_p * n_fav / (n * n_p_fav)
w_p_unfav = n_unfav*n_p / (n*n_p_unfav)
w_unp_fav = n_fav*n_unp / (n*n_unp_fav)
w_unp_unfav = n_unfav*n_unp / (n*n_unp_unfav)

dataset['weight'] = 1

dataset.loc[(dataset['race'] == "Privileged") & (dataset['income'] == ">50K"), ['weight']] = w_p_fav
dataset.loc[(dataset['race'] == "Privileged") & (dataset['income'] == "<=50K"), ['weight']] = w_p_unfav
dataset.loc[(dataset['race'] == "Unprivileged") & (dataset['income'] == ">50K"), ['weight']] = w_unp_unfav
dataset.loc[(dataset['race'] == "Unprivileged") & (dataset['income'] == "<=50K"), ['weight']] = w_unp_unfav


# calculando com probabilidade, só uma forma de mostrar que as fórmulas são equivalentes, já que a maioria das fontes mostra essa primeira forma e não explica de onde vêm
# w_p_fav2 = ((n_p / n) *(n_fav / n)) / (n_p_fav / n)
# w_p_unfav2 = ((n_p / n) *(n_unfav / n)) / (n_p_unfav / n)
# w_unp_fav2 = ((n_unp / n) *(n_fav / n)) / (n_unp_fav / n)
# w_unp_unfav2 = ((n_unp / n) *(n_unfav / n)) / (n_unp_unfav / n)

# print(n, n_p, n_unp, n_fav, n_unfav)
# print(n_p_fav, n_p_unfav, n_unp_fav, n_unp_unfav)
# print(w_p_fav, w_p_unfav, w_unp_fav, w_unp_unfav)
# print(w_p_fav2, w_p_unfav2, w_unp_fav2, w_unp_unfav2)

In [4]:
from sklearn.model_selection import train_test_split

x = dataset.drop(['race', 'native-country', 'income', 'education'], axis = 1)
y = dataset['income']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
x_test.drop('weight', axis = 1)

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,gender,capital-gain,capital-loss,hours-per-week
23884,35,Private,196491,13,Never-married,Sales,Not-in-family,Male,0,0,50
44134,42,?,204817,5,Never-married,?,Own-child,Male,0,0,35
40134,63,?,221592,9,Married-civ-spouse,?,Husband,Male,0,0,40
37758,25,Private,305472,12,Never-married,Machine-op-inspct,Own-child,Male,0,0,48
43920,34,Private,379798,14,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...
23353,32,Private,245482,9,Married-civ-spouse,Machine-op-inspct,Husband,Male,0,0,40
22281,79,Local-gov,84616,7,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,7
11899,35,Private,210310,9,Married-civ-spouse,Sales,Husband,Male,0,0,55
24981,40,State-gov,34218,9,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40


In [5]:
categoricalLabels = (x_train.select_dtypes(include = ['object'])).columns

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', sparse = False, categories = "auto", dtype='int8') # Criando um Codificador
    
OHtrain = pd.DataFrame(encoder.fit_transform(x_train[categoricalLabels])) # Fazendo o treino de codificação inicial
columns = encoder.get_feature_names_out(categoricalLabels) # Para manter o nome das Features no OneHotEncoder eu capturo o nome das Features aqui
OHtrain.columns = columns # Mudo as Colunas para os Nomes capturados antes

OHtest = pd.DataFrame(encoder.transform(x_test[categoricalLabels])) # Repete o processo pro test
columns = encoder.get_feature_names_out(categoricalLabels)
OHtest.columns = columns

OHtrain.index = x_train.index # Atualizando os indices
OHtest.index = x_test.index

old_x_train = x_train[categoricalLabels] # Salva as Features antes do Pré-Processamento para utilizar depois na construção do gráfico
old_x_test = x_test[categoricalLabels]

num_x_train = x_train.drop(categoricalLabels, axis = 1) # Apaga as Features que serão substituidas pelas colunas codificadas
num_x_test = x_test.drop(categoricalLabels, axis = 1)

num_x_train = pd.concat([num_x_train, OHtrain], axis = 1) # Junta o DataSet com as novas Features codificadas
num_x_test = pd.concat([num_x_test, OHtest], axis = 1)
    
num_y_train = y_train.apply(lambda p: 1 if p == '>50K' else 0) # Codifica o Y em Booleano, mantive
num_y_test = y_test.apply(lambda p: 1 if p == '>50K' else 0)

In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression

rgrs = LogisticRegression(random_state = 1)

weights = num_x_train['weight']

rgrs.fit(num_x_train, num_y_train, sample_weight = weights)
previsao = rgrs.predict(num_x_test)
acuraciaNP = mean_absolute_error(num_y_test, previsao)
print(acuraciaNP)

indices = num_x_test.index
races = pd.DataFrame()
races['race'] = dataset['race'].loc[indices]
races['previsao'] = previsao
print()

count_p_fav = races[(races['race'] == 'Privileged') & (races['previsao'] == 0)].shape[0]
count_p = races[races['race'] == 'Privileged'].shape[0]
count_unp_fav = races[(races['race'] == 'Unprivileged') & (races['previsao'] == 0)].shape[0]
count_unp = races[races['race'] == 'Unprivileged'].shape[0]

print(count_p_fav/count_p * 100)
print(count_unp_fav/count_unp * 100)
#n_p_fav/n_p == n_unp_fav/n_unp
#terminar e depois dar uma melhorada
#a ideia agora é melhorar o classificador, vou ver alguns foruns do kaggle que falem sobre.

0.2031665870470211

91.11996811478676
93.50094876660341
